# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Week 9 Session 1: Linear Models for Regression
**Instructor:** Amir Charkhi | **Goal:** Master Linear Regression and Its Variants

### Learning Objectives
- Understand linear regression and its mathematical foundation
- Learn regularization techniques: Ridge, Lasso, and ElasticNet
- Master the complete ML workflow: data loading → EDA → training → evaluation
- Apply cross-validation and hyperparameter tuning
- Interpret model coefficients and feature importance
- Compare model performance using appropriate metrics

---

## 1. Import Libraries

**What you need to do:**  
Import all necessary libraries for data manipulation, visualization, and machine learning.

**Required imports:**
- NumPy and Pandas for data handling
- Matplotlib and Seaborn for visualization
- Scikit-learn for models, preprocessing, and evaluation

**💡 Hint:** We'll need `LinearRegression`, `Ridge`, `Lasso`, `ElasticNet`, `cross_val_score`, `GridSearchCV`, and regression metrics.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ All libraries imported successfully!")

---
## 2. Load the Dataset

**What you need to do:**  
Load the Online Retail dataset - a real-world e-commerce transaction dataset from UCI.

**Theory:**  
This dataset contains actual transactions from a UK-based online retailer between 2009-2011. It includes:
- **InvoiceNo**: Unique transaction identifier
- **StockCode**: Product code
- **Description**: Product name
- **Quantity**: Number of items purchased
- **InvoiceDate**: Transaction date and time
- **UnitPrice**: Price per item
- **CustomerID**: Unique customer identifier
- **Country**: Customer's country

**Our Goal:** Predict **TotalSales** (Quantity × UnitPrice) based on various features we'll engineer.

**💡 Hint:** We'll load the data and create meaningful features for regression modeling.

In [ ]:
# Load the Online Retail dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'

print("📥 Loading Online Retail dataset from UCI...")
print("This may take a minute...\n")

# Load the data
df_raw = pd.read_excel(url)

print(f"✅ Dataset loaded successfully!")
print(f"📊 Shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")

---
## 3. Initial Data Inspection & Feature Engineering

**What you need to do:**  
Inspect the raw data and create meaningful features for regression modeling.

**Tasks:**
1. Display first few rows and check data types
2. Check for missing values
3. Create target variable: **TotalSales** = Quantity × UnitPrice
4. Engineer features from the transaction data
5. Clean the data (remove cancellations, negative values, missing customers)

**💡 Hint:** We'll aggregate data at the transaction (Invoice) level to create our regression dataset.

In [ ]:
# Inspect raw data
print("📋 First few rows of raw data:")
print(df_raw.head())
print("\n" + "="*80 + "\n")

print("🔍 Data Info:")
print(df_raw.info())
print("\n" + "="*80 + "\n")

print("❓ Missing Values:")
print(df_raw.isnull().sum())

In [ ]:
# Data Cleaning and Feature Engineering
print("🧹 Cleaning and engineering features...\n")

# Create a copy for processing
df = df_raw.copy()

# Remove rows with missing CustomerID (we need customers for aggregation)
df = df.dropna(subset=['CustomerID'])

# Remove cancellations (InvoiceNo starting with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Remove negative quantities and prices
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Create TotalSales column
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

# Extract time-based features
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
df['Hour'] = df['InvoiceDate'].dt.hour

print(f"✅ Cleaned dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\n📊 After cleaning: {df.shape[0]:,} transactions remaining")

In [ ]:
# Feature Engineering at Invoice Level
print("🔨 Engineering features...\\n")

invoice_features = df.groupby('InvoiceNo').agg({
    'TotalSales': 'sum',
    'Quantity': 'sum',
    'UnitPrice': 'mean',
    'StockCode': 'nunique',
    'CustomerID': 'first',
    'Country': 'first',
    'Year': 'first',
    'Month': 'first',
    'DayOfWeek': 'first',
    'Hour': 'first'
}).reset_index()

invoice_features.rename(columns={
    'Quantity': 'TotalItems',
    'UnitPrice': 'AvgItemPrice',
    'StockCode': 'NumUniqueProducts'
}, inplace=True)

invoice_features['AvgPricePerItem'] = invoice_features['TotalSales'] / invoice_features['TotalItems']

# Country encoding
top_countries = invoice_features['Country'].value_counts().head(5).index.tolist()
invoice_features['Country_Group'] = invoice_features['Country'].apply(
    lambda x: x if x in top_countries else 'Other'
)
country_dummies = pd.get_dummies(invoice_features['Country_Group'], prefix='Country', drop_first=True)
invoice_features = pd.concat([invoice_features, country_dummies], axis=1)

invoice_features['IsWeekend'] = (invoice_features['DayOfWeek'] >= 5).astype(int)

# ⚠️ CRITICAL: Drop string columns NOW before train-test split
invoice_features = invoice_features.drop(columns=['InvoiceNo', 'Country', 'Country_Group', 'CustomerID'])

print(f"✅ Created {invoice_features.shape[0]:,} invoice-level samples")
print(f"📊 Total features: {invoice_features.shape[1]} columns")
print(f"\n📋 Columns remaining: {list(invoice_features.columns)}")

# Verify all columns except target are numeric
non_target_cols = [col for col in invoice_features.columns if col != 'TotalSales']
numeric_check = invoice_features[non_target_cols].select_dtypes(include=[np.number]).shape[1] == len(non_target_cols)
print(f"\n✅ All feature columns are numeric: {numeric_check}")

In [ ]:
# Display sample of engineered features
print("🎯 Sample of engineered features:")
print(invoice_features.head(10))
print("\n" + "="*80 + "\n")

print("📈 Descriptive statistics:")
print(invoice_features.describe())

---
## 4. Train-Validation-Test Split

**⚠️ CRITICAL: Split BEFORE detailed EDA to prevent data leakage!**

**What you need to do:**  
Split the data into three sets:
- **Training set (60%)**: For model training
- **Validation set (20%)**: For model selection and hyperparameter tuning
- **Test set (20%)**: For final evaluation (DO NOT TOUCH until the end!)

**Theory:**  
The test set represents unseen data in production. It must remain isolated from all training decisions.

**💡 Hint:** Select relevant features and create X (features) and y (target) matrices.

In [ ]:
# Select features for modeling - EXCLUDE original string columns
feature_cols = [
    'TotalItems', 'AvgItemPrice', 'NumUniqueProducts', 
    'AvgPricePerItem', 'Year', 'Month', 'DayOfWeek', 
    'Hour', 'IsWeekend'
]

# Add country dummy variables
country_cols = [col for col in invoice_features.columns if col.startswith('Country_')]
feature_cols.extend(country_cols)

# Create feature matrix X and target vector y
X = invoice_features[feature_cols].copy()
y = invoice_features['TotalSales'].copy()

print(f"🎯 Features for modeling: {len(feature_cols)} features")
print(f"Feature list: {feature_cols}")
print(f"\n📊 X shape: {X.shape}")
print(f"📊 y shape: {y.shape}")

# Verify all columns are numeric
print(f"\n✅ All features are numeric: {X.select_dtypes(include=[np.number]).shape[1] == X.shape[1]}")

In [ ]:
# Split data: 60% train, 20% validation, 20% test
print("✂️ Splitting data into train/validation/test sets...\n")

# First split: 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Second split: 75% of temp = 60% train, 25% of temp = 20% val
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42
)

print(f"✅ Data split complete!")
print(f"\n📊 Training set:   {X_train.shape[0]:>6,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"📊 Validation set: {X_val.shape[0]:>6,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"📊 Test set:       {X_test.shape[0]:>6,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\n🔒 Test set is now LOCKED until final evaluation!")

---
## 5. Exploratory Data Analysis (EDA)

**⚠️ IMPORTANT: Perform EDA ONLY on the training set!**

**What you need to do:**  
Understand the training data through visualization and statistics.

**Tasks:**
1. Summary statistics
2. Target variable distribution
3. Correlation heatmap
4. Top correlated features
5. Scatter plots

**💡 Hint:** This helps us understand relationships before modeling.

In [ ]:
# Summary statistics of training features
print("📊 Training Set Summary Statistics:")
print("="*80)
print(X_train.describe())
print("\n" + "="*80)
print(f"\n🎯 Target Variable (TotalSales) Statistics:")
print(y_train.describe())

In [ ]:
# Target variable distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Original scale
axes[0].hist(y_train, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Total Sales ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Target Variable Distribution (Training Set)')
axes[0].axvline(y_train.mean(), color='red', linestyle='--', label=f'Mean: ${y_train.mean():.2f}')
axes[0].axvline(y_train.median(), color='green', linestyle='--', label=f'Median: ${y_train.median():.2f}')
axes[0].legend()

# Log scale (better for skewed distributions)
axes[1].hist(np.log1p(y_train), bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('Log(Total Sales + 1)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Target Variable Distribution - Log Scale')

plt.tight_layout()
plt.show()

print(f"\n💡 Insight: The target variable is right-skewed (common in sales data).")
print(f"This suggests we might benefit from log transformation or robust models.")

In [ ]:
# Combine features and target for correlation analysis
train_data = X_train.copy()
train_data['TotalSales'] = y_train.values

# Calculate correlations with target (numeric columns only)
correlations = train_data.select_dtypes(include=[np.number]).corr()['TotalSales'].drop('TotalSales').sort_values(ascending=False)

print("📊 Top 10 Features Correlated with Total Sales:")
print("="*60)
for feature, corr in correlations.head(10).items():
    print(f"{feature:.<45} {corr:>8.4f}")

print("\n📊 Bottom 5 Features (Negative/Weak Correlation):")
print("="*60)
for feature, corr in correlations.tail(5).items():
    print(f"{feature:.<45} {corr:>8.4f}")

In [ ]:
# Correlation heatmap (top features only for readability)
top_features = correlations.head(8).index.tolist() + ['TotalSales']
correlation_matrix = train_data[top_features].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Heatmap: Top Features vs Total Sales', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots for top correlated features
top_3_features = correlations.head(3).index.tolist()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, feature in enumerate(top_3_features):
    axes[idx].scatter(X_train[feature], y_train, alpha=0.3, s=10)
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Total Sales ($)')
    axes[idx].set_title(f'{feature} vs Total Sales\n(Correlation: {correlations[feature]:.3f})')
    
    # Add trend line
    z = np.polyfit(X_train[feature], y_train, 1)
    p = np.poly1d(z)
    axes[idx].plot(X_train[feature], p(X_train[feature]), "r--", alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

---
## 6. Feature Scaling

**Theory:**  
Linear models with regularization (Ridge, Lasso, ElasticNet) are sensitive to feature scales. Features with larger magnitudes can dominate the model. **Standardization** (z-score normalization) ensures all features have mean=0 and std=1.

**Formula:** $X_{scaled} = \frac{X - \mu}{\sigma}$

**What you need to do:**  
Fit StandardScaler on training data and transform all splits.

**⚠️ CRITICAL:** Fit the scaler ONLY on training data to prevent data leakage!

**💡 Hint:** Plain LinearRegression doesn't require scaling, but we'll scale for consistency.

In [ ]:
# Initialize and fit scaler on training data ONLY
print("⚖️ Scaling features for regularized models...\n")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)  # For later use

print("✅ Features scaled successfully!")
print(f"\n📊 Scaled training features - Mean: {X_train_scaled.mean():.6f}, Std: {X_train_scaled.std():.6f}")
print(f"\n💡 Note: We fit the scaler only on training data to prevent data leakage.")

---
## 7. Model 1: Linear Regression (Ordinary Least Squares)

**📚 Theory:**  
Linear Regression finds the best-fitting hyperplane through the data by minimizing the **sum of squared errors** (SSE).

**Mathematical Form:**
$$\hat{y} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n$$

**Objective Function (Cost Function):**
$$\text{Minimize: } SSE = \sum_{i=1}^{m} (y_i - \hat{y}_i)^2$$

**Pros:**
- Fast to train
- Highly interpretable (coefficients show feature importance)
- No hyperparameters to tune
- Works well when features have true linear relationships

**Cons:**
- Assumes linear relationships
- Sensitive to outliers
- Prone to overfitting with many features (high variance)
- No built-in feature selection

**When to Use:**
- As a baseline model
- When interpretability is crucial
- When you have fewer features than samples
- When features are not highly correlated (no multicollinearity)

**📖 References:**
- [Scikit-learn: Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- [ISL Book - Chapter 3: Linear Regression](https://www.statlearning.com/)

---

In [ ]:
# Train Linear Regression model
print("🚀 Training Linear Regression (OLS) model...\n")

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)  # No need for scaling for OLS

print("✅ Model trained successfully!")
print(f"\n📊 Model Coefficients (Top 10 by magnitude):")
print("="*60)

# Display coefficients
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lr_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(coef_df.head(10).to_string(index=False))
print(f"\n📍 Intercept: {lr_model.intercept_:.4f}")

In [ ]:
# Make predictions on validation set
y_pred_lr = lr_model.predict(X_val)

# Calculate metrics
lr_rmse = np.sqrt(mean_squared_error(y_val, y_pred_lr))
lr_mae = mean_absolute_error(y_val, y_pred_lr)
lr_r2 = r2_score(y_val, y_pred_lr)

print("📊 Linear Regression - Validation Set Performance:")
print("="*60)
print(f"Root Mean Squared Error (RMSE): ${lr_rmse:>12,.2f}")
print(f"Mean Absolute Error (MAE):      ${lr_mae:>12,.2f}")
print(f"R² Score:                       {lr_r2:>12.4f}")
print("="*60)

print(f"\n💡 Interpretation:")
print(f"   - R² = {lr_r2:.2%}: The model explains {lr_r2:.1%} of variance in Total Sales")
print(f"   - MAE = ${lr_mae:.2f}: On average, predictions are off by ${lr_mae:.2f}")
print(f"   - RMSE = ${lr_rmse:.2f}: The model has higher penalty for large errors")

### Feature Importance Visualization

In [ ]:
# Visualize feature importance (absolute coefficient values)
plt.figure(figsize=(10, 8))
coef_df_top15 = coef_df.head(15).sort_values('Coefficient')

plt.barh(coef_df_top15['Feature'], coef_df_top15['Coefficient'])
plt.xlabel('Coefficient Value')
plt.title('Linear Regression: Top 15 Feature Coefficients', fontsize=14, pad=20)
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

print("💡 Positive coefficients increase sales, negative coefficients decrease sales.")

### Cross-Validation

In [ ]:
# Perform 5-fold cross-validation on training data
print("🔄 Performing 5-Fold Cross-Validation on Linear Regression...\n")

cv_scores_lr = cross_val_score(
    lr_model, X_train, y_train, 
    cv=5, 
    scoring='neg_root_mean_squared_error'
)
cv_scores_lr = -cv_scores_lr  # Convert to positive RMSE

print("📊 Cross-Validation Results (RMSE):")
print("="*60)
for fold, score in enumerate(cv_scores_lr, 1):
    print(f"Fold {fold}: ${score:,.2f}")
print("="*60)
print(f"Mean CV RMSE:   ${cv_scores_lr.mean():,.2f} (± ${cv_scores_lr.std():.2f})")
print(f"Validation RMSE: ${lr_rmse:,.2f}")
print("\n💡 CV scores help us understand model stability across different data splits.")

---
## 8. Model 2: Ridge Regression (L2 Regularization)

**📚 Theory:**  
Ridge Regression adds a **penalty term** to the cost function to prevent overfitting. It shrinks coefficients but keeps all features.

**Cost Function:**
$$\text{Minimize: } SSE + \alpha \sum_{j=1}^{n} \beta_j^2$$

Where:
- $\alpha$ = Regularization strength (hyperparameter)
- $\sum \beta_j^2$ = L2 penalty (sum of squared coefficients)

**Key Characteristics:**
- **L2 Penalty:** Penalizes large coefficients
- **Feature shrinkage:** Reduces coefficient magnitudes (but never to exactly zero)
- **Handles multicollinearity:** Works well when features are correlated
- **Hyperparameter $\alpha$:** Higher $\alpha$ = stronger regularization = simpler model

**Pros:**
- Reduces overfitting
- Handles multicollinearity better than OLS
- More stable than OLS with correlated features
- Keeps all features (doesn't eliminate any)

**Cons:**
- Less interpretable than OLS (shrunk coefficients)
- Doesn't perform feature selection (all features retained)
- Requires hyperparameter tuning
- Requires feature scaling

**When to Use:**
- When you have many features
- When features are correlated (multicollinearity)
- When you want to keep all features but reduce overfitting
- When OLS shows signs of overfitting

**Typical $\alpha$ Values:**
- $\alpha = 0$: Equivalent to OLS
- $\alpha \in [0.01, 100]$: Common range for experimentation
- Larger $\alpha$: More regularization, simpler model

**📖 References:**
- [Scikit-learn: Ridge Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
- [ISL Book - Chapter 6: Ridge Regression](https://www.statlearning.com/)

---

In [ ]:
# Train Ridge Regression with default alpha
print("🚀 Training Ridge Regression (L2 Regularization)...\n")

ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train_scaled, y_train)

print("✅ Ridge model trained with α = 1.0")
print(f"\n📊 Ridge Coefficients (Top 10 by magnitude):")
print("="*60)

ridge_coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Ridge_Coef': ridge_model.coef_,
    'OLS_Coef': lr_model.coef_
}).sort_values('Ridge_Coef', key=abs, ascending=False)

print(ridge_coef_df.head(10).to_string(index=False))
print(f"\n💡 Notice: Ridge coefficients are smaller (shrunk) compared to OLS.")

In [ ]:
# Evaluate Ridge on validation set
y_pred_ridge = ridge_model.predict(X_val_scaled)

ridge_rmse = np.sqrt(mean_squared_error(y_val, y_pred_ridge))
ridge_mae = mean_absolute_error(y_val, y_pred_ridge)
ridge_r2 = r2_score(y_val, y_pred_ridge)

print("📊 Ridge Regression - Validation Set Performance:")
print("="*60)
print(f"Root Mean Squared Error (RMSE): ${ridge_rmse:>12,.2f}")
print(f"Mean Absolute Error (MAE):      ${ridge_mae:>12,.2f}")
print(f"R² Score:                       {ridge_r2:>12.4f}")
print("="*60)

### Hyperparameter Tuning: Ridge

In [ ]:
# Hyperparameter tuning with GridSearchCV
print("🎯 Tuning Ridge hyperparameter (alpha) with GridSearchCV...\n")

param_grid_ridge = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10, 100, 1000]
}

ridge_grid = GridSearchCV(
    Ridge(random_state=42),
    param_grid_ridge,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

ridge_grid.fit(X_train_scaled, y_train)

print(f"\n✅ Best parameters: {ridge_grid.best_params_}")
print(f"📊 Best CV RMSE: ${-ridge_grid.best_score_:,.2f}")

In [ ]:
# Evaluate best Ridge model
best_ridge = ridge_grid.best_estimator_
y_pred_ridge_tuned = best_ridge.predict(X_val_scaled)

ridge_tuned_rmse = np.sqrt(mean_squared_error(y_val, y_pred_ridge_tuned))
ridge_tuned_mae = mean_absolute_error(y_val, y_pred_ridge_tuned)
ridge_tuned_r2 = r2_score(y_val, y_pred_ridge_tuned)

print("📊 Ridge (Tuned) - Validation Set Performance:")
print("="*60)
print(f"Root Mean Squared Error (RMSE): ${ridge_tuned_rmse:>12,.2f}")
print(f"Mean Absolute Error (MAE):      ${ridge_tuned_mae:>12,.2f}")
print(f"R² Score:                       {ridge_tuned_r2:>12.4f}")
print("="*60)
print(f"\n💡 Best alpha = {ridge_grid.best_params_['alpha']}: This is the optimal regularization strength.")

In [ ]:
# Visualize alpha vs performance
results_df = pd.DataFrame(ridge_grid.cv_results_)

plt.figure(figsize=(10, 6))
plt.plot(results_df['param_alpha'], -results_df['mean_test_score'], marker='o', linewidth=2, markersize=8)
plt.xscale('log')
plt.xlabel('Alpha (Regularization Strength)', fontsize=12)
plt.ylabel('Cross-Validated RMSE ($)', fontsize=12)
plt.title('Ridge Regression: Alpha vs Performance', fontsize=14, pad=20)
plt.axvline(ridge_grid.best_params_['alpha'], color='red', linestyle='--', 
            label=f"Best Alpha = {ridge_grid.best_params_['alpha']}")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 The curve shows how different alpha values affect model performance.")
print("   Lower alpha = less regularization (closer to OLS)")
print("   Higher alpha = more regularization (simpler model)")

---
## 9. Model 3: Lasso Regression (L1 Regularization)

**📚 Theory:**  
Lasso (**L**east **A**bsolute **S**hrinkage and **S**election **O**perator) adds an L1 penalty that can **shrink coefficients to exactly zero**, effectively performing **automatic feature selection**.

**Cost Function:**
$$\text{Minimize: } SSE + \alpha \sum_{j=1}^{n} |\beta_j|$$

Where:
- $\alpha$ = Regularization strength
- $\sum |\beta_j|$ = L1 penalty (sum of absolute coefficients)

**Key Characteristics:**
- **L1 Penalty:** Penalizes absolute values of coefficients
- **Sparse solutions:** Sets some coefficients to exactly zero
- **Automatic feature selection:** Eliminates irrelevant features
- **Creates simpler, more interpretable models**

**Pros:**
- Performs automatic feature selection
- Creates sparse models (many coefficients = 0)
- More interpretable than Ridge (fewer features)
- Works well when only some features are important

**Cons:**
- Can be unstable with highly correlated features
- Arbitrarily picks one feature from correlated group
- Requires hyperparameter tuning
- Requires feature scaling

**When to Use:**
- When you suspect many features are irrelevant
- When you want automatic feature selection
- When interpretability is important (fewer features)
- When you need a sparse model

**Ridge vs Lasso:**
- **Ridge:** Keeps all features, shrinks coefficients toward zero
- **Lasso:** Eliminates features, sets coefficients to exactly zero
- **Ridge:** Better when most features are useful
- **Lasso:** Better when many features are irrelevant

**📖 References:**
- [Scikit-learn: Lasso Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
- [ISL Book - Chapter 6: Lasso Regression](https://www.statlearning.com/)

---

In [ ]:
# Train Lasso Regression
print("🚀 Training Lasso Regression (L1 Regularization)...\n")

lasso_model = Lasso(alpha=1.0, random_state=42, max_iter=10000)
lasso_model.fit(X_train_scaled, y_train)

# Count non-zero coefficients
n_nonzero = np.sum(lasso_model.coef_ != 0)

print(f"✅ Lasso model trained with α = 1.0")
print(f"\n🎯 Feature Selection Results:")
print(f"   Total features: {len(lasso_model.coef_)}")
print(f"   Non-zero coefficients: {n_nonzero}")
print(f"   Features eliminated: {len(lasso_model.coef_) - n_nonzero}")

print(f"\n📊 Lasso Coefficients (Non-Zero Features):")
print("="*60)

lasso_coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Lasso_Coef': lasso_model.coef_
}).query('Lasso_Coef != 0').sort_values('Lasso_Coef', key=abs, ascending=False)

print(lasso_coef_df.to_string(index=False))

In [ ]:
# Evaluate Lasso on validation set
y_pred_lasso = lasso_model.predict(X_val_scaled)

lasso_rmse = np.sqrt(mean_squared_error(y_val, y_pred_lasso))
lasso_mae = mean_absolute_error(y_val, y_pred_lasso)
lasso_r2 = r2_score(y_val, y_pred_lasso)

print("📊 Lasso Regression - Validation Set Performance:")
print("="*60)
print(f"Root Mean Squared Error (RMSE): ${lasso_rmse:>12,.2f}")
print(f"Mean Absolute Error (MAE):      ${lasso_mae:>12,.2f}")
print(f"R² Score:                       {lasso_r2:>12.4f}")
print("="*60)

### Hyperparameter Tuning: Lasso

In [ ]:
# Hyperparameter tuning with GridSearchCV
print("🎯 Tuning Lasso hyperparameter (alpha) with GridSearchCV...\n")

param_grid_lasso = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10, 100]
}

lasso_grid = GridSearchCV(
    Lasso(random_state=42, max_iter=10000),
    param_grid_lasso,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

lasso_grid.fit(X_train_scaled, y_train)

print(f"\n✅ Best parameters: {lasso_grid.best_params_}")
print(f"📊 Best CV RMSE: ${-lasso_grid.best_score_:,.2f}")

In [ ]:
# Evaluate best Lasso model
best_lasso = lasso_grid.best_estimator_
y_pred_lasso_tuned = best_lasso.predict(X_val_scaled)

lasso_tuned_rmse = np.sqrt(mean_squared_error(y_val, y_pred_lasso_tuned))
lasso_tuned_mae = mean_absolute_error(y_val, y_pred_lasso_tuned)
lasso_tuned_r2 = r2_score(y_val, y_pred_lasso_tuned)

n_nonzero_tuned = np.sum(best_lasso.coef_ != 0)

print("📊 Lasso (Tuned) - Validation Set Performance:")
print("="*60)
print(f"Root Mean Squared Error (RMSE): ${lasso_tuned_rmse:>12,.2f}")
print(f"Mean Absolute Error (MAE):      ${lasso_tuned_mae:>12,.2f}")
print(f"R² Score:                       {lasso_tuned_r2:>12.4f}")
print("="*60)
print(f"\n🎯 Feature Selection with Best Alpha ({lasso_grid.best_params_['alpha']}):")
print(f"   Selected features: {n_nonzero_tuned}/{len(best_lasso.coef_)}")

# Show selected features
selected_features = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': best_lasso.coef_
}).query('Coefficient != 0').sort_values('Coefficient', key=abs, ascending=False)

print(f"\n📋 Selected Features:")
print(selected_features.to_string(index=False))

---
## 10. Model 4: ElasticNet (L1 + L2 Regularization)

**📚 Theory:**  
ElasticNet combines the **best of both Ridge and Lasso** by using both L1 and L2 penalties.

**Cost Function:**
$$\text{Minimize: } SSE + \alpha \rho \sum_{j=1}^{n} |\beta_j| + \frac{\alpha(1-\rho)}{2} \sum_{j=1}^{n} \beta_j^2$$

Where:
- $\alpha$ = Overall regularization strength
- $\rho$ (l1_ratio) = Balance between L1 and L2 (0 to 1)
  - $\rho = 0$: Pure Ridge
  - $\rho = 1$: Pure Lasso
  - $0 < \rho < 1$: Combination

**Key Characteristics:**
- **Hybrid approach:** Combines L1 sparsity with L2 stability
- **Two hyperparameters:** $\alpha$ and $\rho$
- **Feature selection:** Like Lasso, but more stable
- **Handles correlated features:** Better than Lasso

**Pros:**
- Gets benefits of both Ridge and Lasso
- More stable than Lasso with correlated features
- Can select groups of correlated features (unlike Lasso)
- Performs feature selection (unlike Ridge)

**Cons:**
- Two hyperparameters to tune (more complex)
- Computationally more expensive
- Less interpretable than pure Lasso or Ridge

**When to Use:**
- When you have many correlated features
- When Lasso is too unstable
- When you want feature selection AND stability
- When you're unsure whether to use Ridge or Lasso

**Recommended l1_ratio Values:**
- 0.5: Balanced combination (common default)
- 0.9: More Lasso-like (more feature selection)
- 0.1: More Ridge-like (less feature selection)

**📖 References:**
- [Scikit-learn: ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)
- [Original Paper: Zou & Hastie (2005)](https://www.jstor.org/stable/3647580)

---

In [ ]:
# Train ElasticNet
print("🚀 Training ElasticNet (L1 + L2 Regularization)...\n")

elastic_model = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42, max_iter=10000)
elastic_model.fit(X_train_scaled, y_train)

n_nonzero_elastic = np.sum(elastic_model.coef_ != 0)

print(f"✅ ElasticNet trained with α = 1.0, l1_ratio = 0.5")
print(f"\n🎯 Feature Selection Results:")
print(f"   Selected features: {n_nonzero_elastic}/{len(elastic_model.coef_)}")

# Evaluate on validation set
y_pred_elastic = elastic_model.predict(X_val_scaled)

elastic_rmse = np.sqrt(mean_squared_error(y_val, y_pred_elastic))
elastic_mae = mean_absolute_error(y_val, y_pred_elastic)
elastic_r2 = r2_score(y_val, y_pred_elastic)

print("\n📊 ElasticNet - Validation Set Performance:")
print("="*60)
print(f"Root Mean Squared Error (RMSE): ${elastic_rmse:>12,.2f}")
print(f"Mean Absolute Error (MAE):      ${elastic_mae:>12,.2f}")
print(f"R² Score:                       {elastic_r2:>12.4f}")
print("="*60)

### Hyperparameter Tuning: ElasticNet

In [ ]:
# Hyperparameter tuning with GridSearchCV (alpha AND l1_ratio)
print("🎯 Tuning ElasticNet hyperparameters with GridSearchCV...\n")
print("⏳ This may take longer (2 parameters to tune)...\n")

param_grid_elastic = {
    'alpha': [0.01, 0.1, 1.0, 10],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}

elastic_grid = GridSearchCV(
    ElasticNet(random_state=42, max_iter=10000),
    param_grid_elastic,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

elastic_grid.fit(X_train_scaled, y_train)

print(f"\n✅ Best parameters: {elastic_grid.best_params_}")
print(f"📊 Best CV RMSE: ${-elastic_grid.best_score_:,.2f}")

In [ ]:
# Evaluate best ElasticNet model
best_elastic = elastic_grid.best_estimator_
y_pred_elastic_tuned = best_elastic.predict(X_val_scaled)

elastic_tuned_rmse = np.sqrt(mean_squared_error(y_val, y_pred_elastic_tuned))
elastic_tuned_mae = mean_absolute_error(y_val, y_pred_elastic_tuned)
elastic_tuned_r2 = r2_score(y_val, y_pred_elastic_tuned)

n_nonzero_elastic_tuned = np.sum(best_elastic.coef_ != 0)

print("📊 ElasticNet (Tuned) - Validation Set Performance:")
print("="*60)
print(f"Root Mean Squared Error (RMSE): ${elastic_tuned_rmse:>12,.2f}")
print(f"Mean Absolute Error (MAE):      ${elastic_tuned_mae:>12,.2f}")
print(f"R² Score:                       {elastic_tuned_r2:>12.4f}")
print("="*60)
print(f"\n🎯 Best Hyperparameters:")
print(f"   Alpha: {elastic_grid.best_params_['alpha']}")
print(f"   L1 Ratio: {elastic_grid.best_params_['l1_ratio']}")
print(f"   Selected features: {n_nonzero_elastic_tuned}/{len(best_elastic.coef_)}")

---
## 11. Model Comparison

**What you need to do:**  
Compare all linear models side-by-side to identify the best performer.

**Tasks:**
1. Create summary table of all models
2. Compare metrics: RMSE, MAE, R²
3. Visualize performance comparison
4. Select best model for final evaluation

In [ ]:
# Create comprehensive comparison table
comparison_df = pd.DataFrame({
    'Model': [
        'Linear Regression (OLS)',
        'Ridge (default)',
        'Ridge (tuned)',
        'Lasso (default)',
        'Lasso (tuned)',
        'ElasticNet (default)',
        'ElasticNet (tuned)'
    ],
    'RMSE': [
        lr_rmse, ridge_rmse, ridge_tuned_rmse,
        lasso_rmse, lasso_tuned_rmse,
        elastic_rmse, elastic_tuned_rmse
    ],
    'MAE': [
        lr_mae, ridge_mae, ridge_tuned_mae,
        lasso_mae, lasso_tuned_mae,
        elastic_mae, elastic_tuned_mae
    ],
    'R²': [
        lr_r2, ridge_r2, ridge_tuned_r2,
        lasso_r2, lasso_tuned_r2,
        elastic_r2, elastic_tuned_r2
    ],
    'Features Used': [
        len(X_train.columns),
        len(X_train.columns),
        len(X_train.columns),
        n_nonzero,
        n_nonzero_tuned,
        n_nonzero_elastic,
        n_nonzero_elastic_tuned
    ]
})

# Sort by RMSE (lower is better)
comparison_df = comparison_df.sort_values('RMSE')

print("\n" + "="*80)
print("📊 LINEAR MODELS COMPARISON - VALIDATION SET PERFORMANCE")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

best_model_name = comparison_df.iloc[0]['Model']
print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   RMSE: ${comparison_df.iloc[0]['RMSE']:,.2f}")
print(f"   R²: {comparison_df.iloc[0]['R²']:.4f}")
print(f"   Features: {comparison_df.iloc[0]['Features Used']:.0f}")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RMSE comparison
axes[0].barh(comparison_df['Model'], comparison_df['RMSE'], color='steelblue')
axes[0].set_xlabel('RMSE ($)', fontsize=11)
axes[0].set_title('Model Comparison: RMSE\n(Lower is Better)', fontsize=12, pad=15)
axes[0].invert_yaxis()

# MAE comparison
axes[1].barh(comparison_df['Model'], comparison_df['MAE'], color='coral')
axes[1].set_xlabel('MAE ($)', fontsize=11)
axes[1].set_title('Model Comparison: MAE\n(Lower is Better)', fontsize=12, pad=15)
axes[1].invert_yaxis()

# R² comparison
axes[2].barh(comparison_df['Model'], comparison_df['R²'], color='seagreen')
axes[2].set_xlabel('R² Score', fontsize=11)
axes[2].set_title('Model Comparison: R²\n(Higher is Better)', fontsize=12, pad=15)
axes[2].invert_yaxis()

plt.tight_layout()
plt.show()

---
## 12. Final Evaluation on Test Set

**⚠️ CRITICAL: This is your ONE AND ONLY test set evaluation!**

**Theory:**  
The test set provides an unbiased estimate of production performance. This is your final report card.

**What you need to do:**  
Evaluate the best model on the held-out test set.

**Tasks:**
1. Select best model from validation performance
2. Make predictions on test set
3. Calculate final metrics
4. Visualize predictions vs actual
5. Analyze residuals

In [ ]:
# Select best model (based on validation RMSE)
if best_model_name == 'Ridge (tuned)':
    final_model = best_ridge
    X_test_final = X_test_scaled
elif best_model_name == 'Lasso (tuned)':
    final_model = best_lasso
    X_test_final = X_test_scaled
elif best_model_name == 'ElasticNet (tuned)':
    final_model = best_elastic
    X_test_final = X_test_scaled
else:
    final_model = lr_model
    X_test_final = X_test

print(f"🏆 Selected Model: {best_model_name}")
print(f"\n🔓 Unlocking test set for final evaluation...\n")

In [ ]:
# Final predictions on test set
y_pred_test = final_model.predict(X_test_final)

# Calculate final metrics
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)

print("\n" + "="*80)
print(f"📊 FINAL TEST SET PERFORMANCE: {best_model_name}")
print("="*80)
print(f"Root Mean Squared Error (RMSE): ${test_rmse:>12,.2f}")
print(f"Mean Absolute Error (MAE):      ${test_mae:>12,.2f}")
print(f"R² Score:                       {test_r2:>12.4f}")
print("="*80)

# Compare to validation performance
val_rmse = comparison_df.iloc[0]['RMSE']
val_r2 = comparison_df.iloc[0]['R²']

print(f"\n🔍 Validation vs Test Comparison:")
print(f"   Validation RMSE: ${val_rmse:,.2f}  →  Test RMSE: ${test_rmse:,.2f}")
print(f"   Validation R²: {val_r2:.4f}  →  Test R²: {test_r2:.4f}")

if abs(test_r2 - val_r2) < 0.05:
    print(f"\n✅ Excellent! Test performance is consistent with validation.")
elif test_r2 < val_r2 - 0.05:
    print(f"\n⚠️ Warning: Test performance is worse than validation. Possible overfitting.")
else:
    print(f"\n🎉 Great! Test performance exceeds validation.")

In [ ]:
# Visualize predictions vs actual values
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot: Predicted vs Actual
axes[0].scatter(y_test, y_pred_test, alpha=0.5, s=20)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Total Sales ($)', fontsize=11)
axes[0].set_ylabel('Predicted Total Sales ($)', fontsize=11)
axes[0].set_title(f'{best_model_name}\nPredictions vs Actual (Test Set)', fontsize=12, pad=15)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals plot
residuals = y_test - y_pred_test
axes[1].scatter(y_pred_test, residuals, alpha=0.5, s=20)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted Total Sales ($)', fontsize=11)
axes[1].set_ylabel('Residuals ($)', fontsize=11)
axes[1].set_title('Residual Plot\n(Should be randomly scattered around zero)', fontsize=12, pad=15)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Good residual plot: Points randomly scattered around zero (no patterns).")
print("   Patterns in residuals suggest the model is missing important relationships.")

In [ ]:
# Residuals distribution
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(residuals.mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: ${residuals.mean():.2f}')
plt.axvline(residuals.median(), color='green', linestyle='--', linewidth=2,
            label=f'Median: ${residuals.median():.2f}')
plt.xlabel('Residuals ($)', fontsize=11)
plt.ylabel('Frequency', fontsize=11)
plt.title('Distribution of Residuals (Test Set)', fontsize=12, pad=15)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📊 Residual Statistics:")
print(f"   Mean: ${residuals.mean():.2f} (should be close to 0)")
print(f"   Std Dev: ${residuals.std():.2f}")
print(f"   Min: ${residuals.min():.2f}")
print(f"   Max: ${residuals.max():.2f}")

---
## 13. Key Takeaways & Model Insights

**What you should have learned:**

### 1️⃣ Linear Model Family
✅ **Linear Regression (OLS)**
- Simple, fast, interpretable baseline
- No regularization = risk of overfitting
- Use when: few features, interpretability matters

✅ **Ridge Regression (L2)**
- Shrinks coefficients, keeps all features
- Good for correlated features
- Use when: all features are relevant

✅ **Lasso Regression (L1)**
- Performs feature selection (sets coefs to zero)
- Creates sparse models
- Use when: many irrelevant features

✅ **ElasticNet (L1 + L2)**
- Best of both worlds
- More stable than Lasso with correlated features
- Use when: unsure between Ridge/Lasso

### 2️⃣ Complete ML Workflow
✅ Proper data splitting (train/val/test)
✅ Feature engineering and EDA
✅ Feature scaling for regularized models
✅ Cross-validation for robust evaluation
✅ Hyperparameter tuning with GridSearchCV
✅ Final test set evaluation

### 3️⃣ Model Selection Criteria
- **Accuracy:** Which model has lowest RMSE/MAE?
- **Simplicity:** How many features does it use?
- **Interpretability:** Can we explain predictions?
- **Stability:** Is performance consistent across CV folds?
- **Generalization:** Does validation match test performance?

### 4️⃣ Real-World Insights
- Linear models work well when relationships are approximately linear
- Feature engineering is crucial (we created meaningful features from raw data)
- Regularization helps prevent overfitting
- Feature scaling matters for regularized models
- Always validate on unseen data (test set)

---

### 📝 Reflection Questions
1. Which linear model performed best on our e-commerce data? Why?
2. How did regularization affect the model coefficients?
3. Which features were most important for predicting sales?
4. How does feature scaling affect regularized models?
5. When would you choose Lasso over Ridge?

---

### 🚀 Next Steps: Week 9 Session 2
**Tree-Based Models:**
- Decision Trees
- Random Forests
- Gradient Boosting (XGBoost, LightGBM)

**Questions to explore:**
- Can non-linear models outperform linear models on this dataset?
- How do tree-based models handle feature interactions?
- What are the trade-offs between linear and tree-based models?

---

**AI Tech Institute** | *Building Tomorrow's AI Engineers Today*